## Add (global) emission data

Once you have a **DELWAQ** model, you may want to update your model in order to add new emission data, add sample locations, use different hydrological forcing data, create and run different scenarios etc.

With HydroMT, you can easily read your model and update one or several components of your model using the **update** function of the command line interface (CLI). Here are the steps and some examples on how to **update and add global emission data to your model**.

All lines in this notebook which starts with ! are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste them in your shell to get more feedback.

### Import packages

In this notebook, we will use some functions of HydroMT to prepare configuration and catalog files and plot the new emission data of the updated model. Here are the libraries to import to realize these steps.

In [ ]:
import numpy as np

In [ ]:
# for plotting
import matplotlib.pyplot as plt
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
proj = ccrs.PlateCarree() # plot projection

In [ ]:
# import hydromt
import hydromt
from hydromt_delwaq import DelwaqModel, DemissionModel

### hydroMT CLI update interface

Using the `hydromt update` API, we can update one or several components of an already existing DELWAQ model. Let's get an overview of the available options:

In [ ]:
# Print the options available from the update command
! hydromt update --help

### Adding global data to the model

Using HydroMT, it is quite easy to add additional emission data from already available global data. Here we will see an example where we **add a cattle map (glw_cattle)** from the [GLW 3](https://dataverse.harvard.edu/dataverse/glw_3) data.

You can see a list of all readily available data in HydroMT in the [docs](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets).

In [ ]:
#Let's see what are the attributes of the glw_cattle data source
data_catalog = hydromt.DataCatalog("artifact_data")
data_catalog.to_dict(["glw_cattle"])

Here we can see that our cattle grid is a raster *.tif* file. The important properties for HydroMT are:

* **uri**: path to where the data is stored.
* **data_type**: `HydroMT DataCatalog type` either *RasterDataset* (gridded data), *GeoDataFrame* (vector data) or *GeoDataset* (point timeseries). Our file is a gridded tif file so *RasterDataset*.
* **driver**: driver used to open the data. Either *rasterio* (GDAL compliant raster file), *raster_xarray* (NetCDF file or zarr file). Our file is a tif raster so *rasterio*.

The rest are just additional **meta** information on the data source.

### Preparing the configuration file

As the **glw_cattle** data is a **raster** data, we will add it to our model using the `[setup_emission_raster]` component of HydroMT Delwaq.

Let's prepare a **HydroMT configuration file** with our options for preparing the cattle emission raster. All available options are available in the [docs(setup_emission_raster)](https://deltares.github.io/hydromt_delwaq/latest/generated/hydromt_delwaq.delwaq.DelwaqModel.setup_emission_raster.html).

In [ ]:
%%writefile delwaq_update_emission_cattle.yml

steps:
  - setup_emission_raster:
      emission_fn: glw_cattle
      scale_method: average
      fillna_method: zero
      area_division: True

Some explanations about the option we chose here:

* **emission_fn**: name of the cattle data source in hydroMT Data Catalog.
* **scale_method**: method for resampling from the data resolution (10km x 10km for GLW) to the model resolution (5km*5km, same as the wflow_piave model resolution). Available methods are *nearest*, *average*, *mode* (for classification).
* **fillna_method**: method to fill NaN values in the data sources either *nearest*, *zero* or *value* to fill with a user defined value.
* **area_division**: if needed do the resampling in head/m2 (True) instead of head (False). The GLW cattle grid presents the number of cows per grid cell. To resample to a different grid resolution, we need to resample cow densities instead. After resampling, the cattle grid will be again in cows per model grid cell.

### Updating the model with the cattle grid

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
! hydromt update demission EM_piave -o ./EM_piave_extended -i delwaq_update_emission_cattle.yml -d artifact_data --fo -vv

The example above means the following: run **hydromt** with:

- `update demission`: i.e. update a demission model
- `EM_piave`: original model folder
- `-o ./EM_piave_extended`: output updated model folder
- `-i delwaq_update_emission_cattle.yml`: hydroMT configuration file containing the components and options to update
- `-d artifact_data`: predefined or local data catalog to use. Here the openly available predefined artifact_data catalog
- `-v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

### Visualization of the new emission map

We can now plot our newly created cattle grid.

In [ ]:
# Load the original and updated model with hydromt
mod = DemissionModel(root='EM_piave_extended', mode='r')

In [ ]:
# Edit the lines below to change the emission map and its colormap
emissionmap = 'glw_cattle'
colormap = 'Reds'

#Load the emission map
da = mod.staticdata.data[emissionmap].raster.mask_nodata()
da.attrs.update(long_name=emissionmap, units='-')

#Plot
# we assume the model maps are in the geographic CRS EPSG:4326
proj = ccrs.PlateCarree()
# adjust zoomlevel and figure size to your basis size & aspect
zoom_level = 10
figsize=(10, 8)

# initialize image with geoaxes
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(projection=proj)
extent = np.array(da.raster.box.buffer(0.02).total_bounds)[[0, 2, 1, 3]]
ax.set_extent(extent, crs=proj)

# add sat background image
ax.add_image(cimgt.QuadtreeTiles(), zoom_level, alpha=0.5)

## plot emission map
cmap = plt.cm.get_cmap(colormap)
kwargs = dict(cmap=cmap)
# plot 'normal' elevation
da.plot(transform=proj, ax=ax, zorder=1, cbar_kwargs=dict(aspect=30, shrink=.8), **kwargs)

ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
ax.set_ylabel(f"latitude [degree north]")
ax.set_xlabel(f"longitude [degree east]")
_ = ax.set_title(f"Delwaq emission map")